In [ ]:
#data_test
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-4JBeNhMN0ouKnMTQ3_d-N8CnPKoRo3q' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-4JBeNhMN0ouKnMTQ3_d-N8CnPKoRo3q" -O '/content/data_test.zip' && rm -rf /tmp/cookies.txt

In [ ]:
#data-train
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1IhQyxq4mKZdUOAz5Dw-_HY0ql3HvuMm4' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1IhQyxq4mKZdUOAz5Dw-_HY0ql3HvuMm4" -O /content/dataset.zip && rm -rf /tmp/cookies.txt

In [ ]:
#!unzip -qq '/content/dataset.zip'

In [ ]:
#data_val
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-1u-7qDEj7v-gG4gU2uQIVLyCXi2X98H' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-1u-7qDEj7v-gG4gU2uQIVLyCXi2X98H" -O '/content/data_val.zip' && rm -rf /tmp/cookies.txt

In [ ]:
#!unzip -qq '/content/data_test.zip'

In [ ]:
#!unzip -qq '/content/data_val.zip'

In [ ]:
#!mv '/content/content/content/data/test' '/content/content/data'

In [ ]:
import cv2
import os
from imutils import face_utils
from tqdm import tqdm
from PIL import Image
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.applications import VGG16
from keras import models
from keras import layers
from numpy import array
from keras.utils.vis_utils import plot_model
from tqdm import tqdm
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [ ]:
CONSTANT_FRAMES = 5

In [ ]:
input = layers.Input(name = 'input', shape=(224, 224, 1))

block1 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block1_1')(input)
block1 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block1_2')(block1)

block2 =  layers.MaxPool2D(pool_size = (2, 2), strides = 2, name = 'block2_1')(block1)
block2 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block2_2')(block2)
block2 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block2_3')(block2)

block3 =  layers.MaxPool2D(pool_size = (2, 2), strides = 2, name = 'block3_1')(block2)
block3 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block3_2')(block3)
block3 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block3_3')(block3)
block3 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block3_4')(block3)

block4 =  layers.MaxPool2D(pool_size = (2, 2), strides = 2, name = 'block4_1')(block3)
block4 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block4_2')(block4)
block4 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block4_3')(block4)
block4 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block4_4')(block4)

block5 =  layers.MaxPool2D(pool_size = (2, 2), strides = 2, name = 'block5_1')(block4)
block5 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block5_2')(block5)
block5 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block5_3')(block5)
block5 =  layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'block5_4')(block5)

block6 =  layers.MaxPool2D((2, 2), name = 'block6_1')(block5)

fc6 = layers.Flatten(name = 'fc6')(block6)
fc6 = layers.Dense(8,activation='softmax')(fc6)

fc5 = layers.Flatten(name = 'fc5')(block5)
fc5 = layers.Dense(8,activation='softmax')(fc5)

block6 =  layers.UpSampling2D((4,4), interpolation = 'bilinear', name = 'up6_x4')(block6)
block5 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up5_x2')(block5)
concat5 = layers.Concatenate(name = 'concat5')([block6, block5, block4])
concat5 = layers.Conv2D(64,(1,1), activation = 'relu')(concat5)
fc4 = layers.Flatten(name = 'fc4')(concat5)
fc4 = layers.Dense(8,activation='softmax')(fc4)

block6 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up6_x8')(block6)
block5 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up5_x4')(block5)
concat4 = layers.Concatenate(name = 'concat4')([block6, block5, block3])
concat4 = layers.Conv2D(32,(1,1), activation = 'relu')(concat4)
fc3 = layers.Flatten(name = 'fc3')(concat4)
fc3 = layers.Dense(8,activation='softmax')(fc3)

block6 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up6_x16')(block6)
block5 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up5_x8')(block5)
block4 =  layers.UpSampling2D((4,4), interpolation = 'bilinear', name = 'up4_x4')(block4)
block3 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up3_x2')(block3)
concat3 = layers.Concatenate(name = 'concat3')([block6, block5, block4, block3, block2])
concat4 = layers.Conv2D(8,(1,1), activation = 'relu')(concat3)
fc2 = layers.Flatten(name = 'fc2')(concat3)
fc2 = layers.Dense(8,activation='softmax')(fc2)

block6 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up6_x32')(block6)
block5 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up5_x16')(block5)
block4 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up4_x8')(block4)
block3 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up3_x4')(block3)
concat2 = layers.Concatenate(name = 'concat2')([block6, block5, block4, block3, block1])
concat2 = layers.Conv2D(8,(1,1), activation = 'relu')(concat2)
fc1 = layers.Flatten(name = 'fc1')(concat2)
fc1 = layers.Dense(8,activation='softmax')(fc1)

block6 =  layers.UpSampling2D((1,1), interpolation = 'bilinear', name = 'up6__x32')(block6)
block5 =  layers.UpSampling2D((1,1), interpolation = 'bilinear', name = 'up5__x16')(block5)
block4 =  layers.UpSampling2D((1,1), interpolation = 'bilinear', name = 'up4__x8')(block4)
block3 =  layers.UpSampling2D((1,1), interpolation = 'bilinear', name = 'up3__x4')(block3)
block2 =  layers.UpSampling2D((2,2), interpolation = 'bilinear', name = 'up2_x2')(block2)
concat1 = layers.Concatenate(name = 'concat1')([block6, block5, block4, block3, block2, block1])
concat1 = layers.Conv2D(8,(1,1), activation = 'relu')(concat1)
fc0 = layers.Flatten(name = 'fc0')(concat1)
fc0 = layers.Dense(8,activation='softmax')(fc0)

output = layers.Average()([fc0, fc1, fc2, fc3, fc4, fc5, fc6])

In [ ]:
model = models.Model(input, output)

In [ ]:
'''
def myprint(s):
    with open('modelsummary.txt','w+') as f:
        print(s, file=f)
#model.summary(print_fn=myprint)
#model.summary(line_length=100)
#x = model.summary()
with open('out.txt', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
'''

"\ndef myprint(s):\n    with open('modelsummary.txt','w+') as f:\n        print(s, file=f)\n#model.summary(print_fn=myprint)\n#model.summary(line_length=100)\n#x = model.summary()\nwith open('out.txt', 'w') as f:\n    model.summary(print_fn=lambda x: f.write(x + '\n'))\n"

In [ ]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#for testing generator
label_test = [i for i in range(2185, 2392)] #test
start_test = {'surprised':2199, 'sad':2232, 'calm':2265, 'disgust':2279, 'angry':2312, 'fearful':2345, 'happy':2378, 'neutral':2392} #test
mapping_test = {}
i = 2185
for key, value in start_test.items():
    while(i<value):
        mapping_test[i] = key
        i += 1
#batch size means video
def imageLoader_test(label_test, batch_size):

    L = len(label_test)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
            X = Dataset_loader_X_test(label_test[batch_start:limit])
            Y = Dataset_loader_Y_test(label_test[batch_start:limit])

            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size  
            batch_end += batch_size
def Dataset_loader_X_test(label_test):
    RESIZE = 224
    DIR = '/content/content/content/data/test/'
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for i in label_test:
        counter = 0
        DIR2 = DIR + mapping_test[i] + '/' + str(i)
        for IMAGE_NAME in (os.listdir(DIR2)):
            if(counter == CONSTANT_FRAMES):
                break
            PATH = os.path.join(DIR2,IMAGE_NAME)
            _, ftype = os.path.splitext(PATH)
            if ftype == ".jpg":
                img = read(PATH)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (RESIZE,RESIZE))
                IMG.append(np.array(img)/255.)
            counter += 1
    IMG = np.array(IMG)
    IMG = IMG.reshape(IMG.shape[0], 224, 224, 1)
    return IMG
def Dataset_loader_Y_test(label_test):
    DIR = '/content/content/content/data/test/'
    y = []
    dict_ = {'angry':1, 'sad':2, 'fearful':3, 'happy':4, 'neutral':5, 'surprised':6, 'disgust':7, 'calm':8}
    for i in label_test:
        counter = 0
        DIR2 = DIR + mapping_test[i] + '/' + str(i)
        for IMAGE_NAME in (os.listdir(DIR2)):
            if(counter == CONSTANT_FRAMES):
                break
            temp = [0, 0, 0, 0, 0, 0, 0, 0]
            temp[dict_[mapping_test[i]]-1] = 1
            y.append(temp)
            counter += 1
    y = np.array(y)
    y = y.reshape(y.shape[0], 8)
    return y

In [ ]:
#for validation generator
label_val = [i for i in range(1690, 2185)] #val
start_val = {'surprised':1730, 'sad':1805, 'calm':1880, 'disgust':1920, 'angry':1995, 'fearful':2070, 'happy':2145, 'neutral':2185} #val
mapping_val = {}
i = 1690
for key, value in start_val.items():
    while(i<value):
        mapping_val[i] = key
        i += 1
#batch size means video
def imageLoader_val(label_val, batch_size):

    L = len(label_val)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
            X = Dataset_loader_X_val(label_val[batch_start:limit])
            Y = Dataset_loader_Y_val(label_val[batch_start:limit])

            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size  
            batch_end += batch_size
def Dataset_loader_X_val(label_val):
    RESIZE = 224
    DIR = '/content/content/data/val/'
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for i in label_val:
        counter = 0
        DIR2 = DIR + mapping_val[i] + '/' + str(i)
        for IMAGE_NAME in (os.listdir(DIR2)):
            if(counter == CONSTANT_FRAMES):
                break
            PATH = os.path.join(DIR2,IMAGE_NAME)
            _, ftype = os.path.splitext(PATH)
            if ftype == ".jpg":
                img = read(PATH)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (RESIZE,RESIZE))
                IMG.append(np.array(img)/255.)
            counter += 1
    IMG = np.array(IMG)
    IMG = IMG.reshape(IMG.shape[0], 224, 224, 1)
    return IMG
def Dataset_loader_Y_val(label_val):
    DIR = '/content/content/data/val/'
    y = []
    dict_ = {'angry':1, 'sad':2, 'fearful':3, 'happy':4, 'neutral':5, 'surprised':6, 'disgust':7, 'calm':8}
    for i in label_val:
        counter = 0
        DIR2 = DIR + mapping_val[i] + '/' + str(i)
        for IMAGE_NAME in (os.listdir(DIR2)):
            if(counter == CONSTANT_FRAMES):
                break
            temp = [0, 0, 0, 0, 0, 0, 0, 0]
            temp[dict_[mapping_val[i]]-1] = 1
            y.append(temp)
            counter += 1
    y = np.array(y)
    y = y.reshape(y.shape[0], 8)
    return y

In [ ]:
#for train
label_train = [i for i in range(0,1690)] #train
start_train = {'neutral':130, 'disgust':260, 'sad':520, 'angry':780, 'fearful':1040, 'surprised':1170, 'calm':1430, 'happy':1690} #train
mapping_train = {}
i = 0
for key, value in start_train.items():
    while(i<value):
        mapping_train[i] = key
        i += 1
#batch size means video
def imageLoader_train(label_train, batch_size):

    L = len(label_train)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
            X = Dataset_loader_X_train(label_train[batch_start:limit])
            Y = Dataset_loader_Y_train(label_train[batch_start:limit])

            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size  
            batch_end += batch_size
            if batch_end%20==0:
              t=batch_end/20
              cc=0
              for key,value in start_train.items():
                cc+=1
                if cc==t:
                  batch_start=start_train[key]
                  batch_end=batch_start+1
                  break
def Dataset_loader_X_train(label_train):
    RESIZE = 224
    DIR = '/content/content/data/train/'
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for i in label_train:
        counter = 0
        DIR2 = DIR + mapping_train[i] + '/' + str(i)
        for IMAGE_NAME in (os.listdir(DIR2)):
            if(counter == CONSTANT_FRAMES):
                break
            PATH = os.path.join(DIR2,IMAGE_NAME)
            _, ftype = os.path.splitext(PATH)
            if ftype == ".jpg":
                img = read(PATH)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (RESIZE,RESIZE))
                IMG.append(np.array(img)/255.)
            counter += 1
    IMG = np.array(IMG)
    IMG = IMG.reshape(IMG.shape[0], 224, 224, 1)
    return IMG
def Dataset_loader_Y_train(label_train):
    DIR = '/content/content/data/train/'
    y = []
    dict_ = {'angry':1, 'sad':2, 'fearful':3, 'happy':4, 'neutral':5, 'surprised':6, 'disgust':7, 'calm':8}
    for i in label_train:
        counter = 0
        DIR2 = DIR + mapping_train[i] + '/' + str(i)
        for IMAGE_NAME in (os.listdir(DIR2)):
            if(counter == CONSTANT_FRAMES):
                break
            temp = [0, 0, 0, 0, 0, 0, 0, 0]
            temp[dict_[mapping_train[i]]-1] = 1
            y.append(temp)
            counter += 1
    y = np.array(y)
    y = y.reshape(y.shape[0], 8)
    return y

In [ ]:
History = model.fit_generator(imageLoader_train(label_train, 1), steps_per_epoch = 160, epochs = 1)

Epoch 1/1
160/160 [==============================] - 448s 3s/step - loss: 14.7078 - accuracy: 0.0875


In [ ]:
result = model.evaluate_generator(imageLoader_test(label_test, 1), steps=1);

In [ ]:
print('Accuracy of given cnn-ds model:-', result[1]*100, '%')

Accuracy of given cnn-ds model:- 39.89911111 %


In [ ]:
import matplotlib.pyplot as plt
plt.plot(History.history['acc'])
plt.plot(History.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
import pickle
with open('model_dscnn', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
!cp '/content/model_dscnn' '/content/drive/My Drive/'

In [ ]:
import pickle
fil = open('/content/model_dscnn', 'rb') 
model_ = pickle.loads(fil.read()) 
fil.close()

In [ ]:
PATH = '/content/frame49.jpg'
read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
X = []
IMG = []
img = read(PATH)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (224,224))
IMG.append(np.array(img)/255.)
IMG = np.array(IMG)
X = IMG.reshape(IMG.shape[0], 224, 224, 1)

In [ ]:
print(X.shape)

(1, 224, 224, 1)


In [ ]:
pred = model_.predict(X)

In [ ]:
print(pred)

[[0. 0. 0. 0. 0. 1. 0. 0.]]
